<a href="https://colab.research.google.com/github/Rapheal-Jones/Image-segementation/blob/master/PSP_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/drive/')

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [ ]:
import cv2
import numpy as np
import os
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ReLU,LeakyReLU
from tensorflow.keras.layers import Add,Activation
from tensorflow.keras.layers import GlobalAveragePooling2D,AveragePooling2D
from tensorflow.keras.layers import UpSampling2D

In [ ]:
#dataset path

train_folder="/drive/My Drive/Project/datasets/train/"
#test_folder="/drive/My Drive/Project/datasets/test/"
validate_folder="/drive/My Drive/Project/datasets/val/"


In [ ]:
#load images
def images(path,k1,k2):
    i=0
    image,label=[],[]
    folder_file=os.listdir(os.path.join(path,dir[0]))
    
    for img in folder_file[k1:k2]:
        
        img_path=os.path.join(path,dir[0],img)
        img_arr=cv2.imread(img_path,cv2.IMREAD_UNCHANGED)
        img_normalize=cv2.normalize(img_arr,None,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        image.append(img_normalize)

        lab_path=os.path.join(path,dir[1],img)
        lab_arr=cv2.imread(lab_path,cv2.IMREAD_COLOR)
        lab_normalize=cv2.normalize(lab_arr,None,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        label.append(lab_normalize)
        
        i+=1
        print(i,end=' ')

    return np.array(image),np.array(label)


In [ ]:
dir=['image','label']

image_train, label_train=images(train_folder,0,500)
print()
#image_test,label_test=images(test_folder)
print()
image_val, label_val=images(validate_folder,0,150)

val_data=(image_val,label_val)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 

KeyboardInterrupt: ignored

In [ ]:
def conv_block(X,filters,block):
    # resiudal block with dilated convolutions
    # add skip connection at last after doing convoluion operation to input X
    
    b = 'Layer_'+block+'_'
    f1,f2,f3 = filters
    X_skip = X
    # block_a
    X = Convolution2D(filters=f1,kernel_size=(1,1),dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',name=b+'a')(X)
    X = BatchNormalization(name=b+'batch_norm_a')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_a')(X)
    # block_b
    X = Convolution2D(filters=f2,kernel_size=(3,3),dilation_rate=(2,2),padding='same',kernel_initializer='he_normal',name=b+'b')(X)
    X = BatchNormalization(name=b+'batch_norm_b')(X)
    X = LeakyReLU(alpha=0.2,name=b+'leakyrelu_b')(X)
    # block_c
    X = Convolution2D(filters=f3,kernel_size=(1,1),dilation_rate=(1,1),padding='same',kernel_initializer='he_normal',name=b+'c')(X)
    X = BatchNormalization(name=b+'batch_norm_c')(X)
    # skip_conv
    X_skip = Convolution2D(filters=f3,kernel_size=(3,3),padding='same',name=b+'skip_conv')(X_skip)
    X_skip = BatchNormalization(name=b+'batch_norm_skip_conv')(X_skip)
    # block_c + skip_conv
    X = Add(name=b+'add')([X,X_skip])
    X = ReLU(name=b+'relu')(X)
    return X

In [ ]:
def base_feature_maps(input_layer):
    # base covolution module to get input image feature maps 
    
    # block_1
    base = conv_block(input_layer,[32,32,64],'1')
    # block_2
    base = conv_block(base,[64,64,128],'2')
    # block_3
    base = conv_block(base,[128,128,256],'3')
    
    return base

In [ ]:
def pyramid_feature_maps(input_layer):
    # pyramid pooling module
    
    base = base_feature_maps(input_layer)
    # red
    red = GlobalAveragePooling2D(name='red_pool')(base)
    red = tf.keras.layers.Reshape((1,1,256))(red)
    red = Convolution2D(filters=64,kernel_size=(1,1),name='red_1_by_1')(red)
    red = UpSampling2D(size=512,interpolation='bilinear',name='red_upsampling')(red)
    # yellow
    yellow = AveragePooling2D(pool_size=(2,2),name='yellow_pool')(base)
    yellow = Convolution2D(filters=64,kernel_size=(1,1),name='yellow_1_by_1')(yellow)
    yellow = UpSampling2D(size=2,interpolation='bilinear',name='yellow_upsampling')(yellow)
    # blue
    blue = AveragePooling2D(pool_size=(4,4),name='blue_pool')(base)
    blue = Convolution2D(filters=64,kernel_size=(1,1),name='blue_1_by_1')(blue)
    blue = UpSampling2D(size=4,interpolation='bilinear',name='blue_upsampling')(blue)
    # green
    green = AveragePooling2D(pool_size=(8,8),name='green_pool')(base)
    green = Convolution2D(filters=64,kernel_size=(1,1),name='green_1_by_1')(green)
    green = UpSampling2D(size=8,interpolation='bilinear',name='green_upsampling')(green)
    # base + red + yellow + blue + green
    
    return tf.keras.layers.concatenate([base,red,yellow,blue,green])

In [ ]:
def last_conv_module(input_layer):
    X = pyramid_feature_maps(input_layer)
    X = Convolution2D(filters=3,kernel_size=3,padding='same',name='last_conv_3_by_3')(X)
    X = BatchNormalization(name='last_conv_3_by_3_batch_norm')(X)
    X = Activation('softmax',name='last_conv_relu')(X)
    return X

In [ ]:

input_layer = tf.keras.Input(shape=(512,512,3),name='input')
output_layer = last_conv_module(input_layer)

model = tf.keras.Model(inputs=input_layer,outputs=output_layer)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
#model=tf.keras.models.load_model("/drive/My Drive/Project/save/images_500")

In [ ]:
model.fit(image_train,label_train,validation_data=val_data,epochs=10,batch_size=2)


Epoch 1/10
250/250 [==============================] - 2561s 10s/step - loss: 0.6457 - accuracy: 0.3160 - val_loss: 0.1311 - val_accuracy: 0.2751
Epoch 2/10
250/250 [==============================] - 2562s 10s/step - loss: 0.6417 - accuracy: 0.3079 - val_loss: 0.1311 - val_accuracy: 0.3057
Epoch 3/10
250/250 [==============================] - 2567s 10s/step - loss: 0.6417 - accuracy: 0.3026 - val_loss: 0.1311 - val_accuracy: 0.4672
Epoch 4/10
250/250 [==============================] - 2561s 10s/step - loss: 0.6417 - accuracy: 0.2977 - val_loss: 0.1310 - val_accuracy: 0.3929
Epoch 5/10
250/250 [==============================] - 2557s 10s/step - loss: 0.6417 - accuracy: 0.2954 - val_loss: 0.1310 - val_accuracy: 0.3879
Epoch 6/10
250/250 [==============================] - 2567s 10s/step - loss: 0.6417 - accuracy: 0.2958 - val_loss: 0.1310 - val_accuracy: 0.3887
Epoch 7/10
250/250 [==============================] - 2565s 10s/step - loss: 0.6417 - accuracy: 0.2992 - val_loss: 0.1310 - val_ac

In [ ]:
model.save("/drive/My Drive/Project/save/images_500")

In [ ]:
#model.evaluate(image_test,label_test)

In [ ]:
#model.summary()